In [73]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import re 
from geopy.distance import geodesic
from datetime import datetime, timedelta, time, date
import matplotlib.pyplot as plt
# Example 5. FAM (Fleet Assignment Model)

In [74]:
# Define the inputs
# Define the inputs

# L: set of flights
flights_df = pd.read_excel('Example_Data_Fam.xlsx', sheet_name='Flight')#.set_index('Flight no.')
flights_list = flights_df['Flight no.'].to_list() 

# K: set of aircraft types
aircraft_df = pd.read_excel('Example_Data_Fam.xlsx', sheet_name='Aircraft').set_index('AC Type')
aircraft = aircraft_df.to_dict(orient='index')
ac_list = list(aircraft.keys())

for k in aircraft:
    aircraft[k]['TAT'] = timedelta(minutes=aircraft[k]['TAT'])

flights = flights_df.merge(aircraft_df.reset_index()[['AC Type', 'Range']], how='cross')

In [75]:
flight_distance = {     # Distance [km]
    'PDL': {'PDL': 0, 'LIS': 1461, 'OPO': 1536, 'FNC': 975, 'YTO': 4545, 'BOS': 3888},
    'LIS': {'PDL': 1461, 'LIS': 0, 'OPO': 336, 'FNC': 973, 'YTO': 5790, 'BOS': 5177},
    'OPO': {'PDL': 1536, 'LIS': 336, 'OPO': 0, 'FNC': 1244, 'YTO': 5671, 'BOS': 5081},
    'FNC': {'PDL': 975, 'LIS': 973, 'OPO': 1244, 'FNC': 0, 'YTO': 5515, 'BOS': 4851},
    'YTO': {'PDL': 4545, 'LIS': 5790, 'OPO': 5671, 'FNC': 5515, 'YTO': 0, 'BOS': 691},
    'BOS': {'PDL': 3888, 'LIS': 5177, 'OPO': 5081, 'FNC': 4851, 'YTO': 691, 'BOS': 0}
}

In [76]:
# List of unique airports from Origin and Destination columns
airports = list(set(flights['From'].unique()).union(set(flights['To'].unique())))

In [77]:
misc_date = date(1,1,1)

flights['Distance'] = flights.apply(lambda row: flight_distance[row['From']][row['To']], axis=1)

# Drop rows with distance > range
unfeasible_flights = flights[flights['Distance'] > flights['Range']]
flights = flights[flights['Distance'] <= flights['Range']].drop(columns=['Range'])
flights['Arrival'] = flights.apply(lambda row: (datetime.combine(misc_date,row['Arrival']) + aircraft[row['AC Type']]['TAT']).time(), axis=1)
flights['Overnight'] = flights.apply(lambda row: row['Arrival'] < row['Departure'], axis=1)

# Print flight: X not feasible with aircraft: Y
print('Flights not feasible with aircraft:')
print(unfeasible_flights[['Flight no.', 'AC Type', 'From', 'To', 'Distance']])
print('\n')

Flights not feasible with aircraft:
   Flight no. AC Type From   To  Distance
15      S4222    A320  PDL  YTO      4545
17      S4221    A320  PDL  BOS      3888
33      S4223    A320  YTO  PDL      4545
35      S422X    A320  BOS  PDL      3888




In [62]:
# Make flights dictionary with main keys: AC Type, with a sub dictionary of flight numbers and each with flight details
flights_dict = {}
for i in flights_list:
    flights_dict[i] = flights[flights['Flight no.'] == i].set_index('AC Type').to_dict(orient='index')

flights = flights_dict

In [63]:
import pandas as pd

# Create an empty list to store the data
data = []
# Iterate over airports, aircraft types, and flights
for l in flights:
    for k in flights[l]:
        if flights[l][k]['Distance'] < aircraft[k]['Range']:
            for n in airports:
                if flights[l][k]['From'] == n:
                    data.append([k, n, l, flights[l][k]['Departure'], flights[l][k]['Distance'], 'Departure'])
                if flights[l][k]['To'] == n:
                    data.append([k, n, l, flights[l][k]['Arrival'], flights[l][k]['Distance'], 'Arrival'])

# Create a dataframe with the data
events = pd.DataFrame(data, columns=['AC Type','Airport', 'Flight N', 'Time', 'Distance', 'D_A'])

# Add the TAT to the arrival times

events.sort_values(by=['AC Type', 'Airport', 'Time'], inplace=True)

# Reset the numbering of the events
events.reset_index(drop=True, inplace=True)

In [64]:
events

,AC Type,Airport,Flight N,Time,Distance,D_A
0,A310,BOS,S422X,03:15:00,3888,Departure
1,A310,BOS,S4221,22:25:00,3888,Arrival
2,A310,FNC,S4160,13:25:00,975,Arrival
3,A310,FNC,S4161,17:05:00,975,Departure
4,A310,LIS,S4128,00:50:00,1461,Arrival
...,...,...,...,...,...,...
59,A320,PDL,S4212,19:00:00,1536,Departure
60,A320,PDL,S4161,19:35:00,975,Arrival
61,A320,PDL,S4129,21:30:00,1461,Arrival
62,A320,PDL,S4213,21:50:00,1536,Arrival


In [65]:
# For each airport and aircraft type i need to create a loop of ground arcs, each starting from the last event and ending at the next event, if there is no more events then the last ground arc is the overnight arc and it ends at the first event of the next day (first event of the next day is the first event of the same airport and aircraft type)

ground_arcs = pd.DataFrame(columns=['AC Type', 'Airport', 'Start Time', 'End Time'])

for k in ac_list:
    for n in airports:
        df = events[(events['AC Type'] == k) & (events['Airport'] == n)].sort_values(by=['Time'])
        for i in range(len(df)):
            if i == 0:
                ground_arcs = pd.concat([ground_arcs, pd.DataFrame({'AC Type': k, 'Airport': n, 'Start Time': [df.iloc[-1]['Time']], 'End Time': [df.iloc[i]['Time']]})], ignore_index=True)
            else:
                ground_arcs = pd.concat([ground_arcs, pd.DataFrame({'AC Type': k, 'Airport': n, 'Start Time': [df.iloc[i-1]['Time']], 'End Time': [df.iloc[i]['Time']]})], ignore_index=True)

ground_arcs.sort_values(by=['AC Type', 'Airport'], inplace=True)
# Drop rows if start time and end time are the same
ground_arcs = ground_arcs[~ground_arcs.apply(lambda row: (row['Start Time'] == row['End Time']), axis=1)]
ground_arcs['Arc ID'] = ground_arcs.groupby(['AC Type','Airport']).cumcount()
# Add the AC Type and Airport to the Arc ID
ground_arcs['Arc ID'] = ground_arcs.apply(lambda row: str(row['AC Type']) + '_' + str(row['Airport']) + '_' + str(row['Arc ID']), axis=1)
ground_arcs['Overnight'] = ground_arcs.apply(lambda row: (row['End Time'] < row['Start Time']), axis=1)
ground_arcs.sort_values(by='Arc ID', inplace=True)

In [66]:
ground_arcs[ground_arcs['AC Type'] == 'A310'].sort_values(by=['AC Type', 'Airport', 'Arc ID'])

,AC Type,Airport,Start Time,End Time,Arc ID,Overnight
8,A310,BOS,22:25:00,03:15:00,A310_BOS_0,True
9,A310,BOS,03:15:00,22:25:00,A310_BOS_1,False
2,A310,FNC,17:05:00,13:25:00,A310_FNC_0,True
3,A310,FNC,13:25:00,17:05:00,A310_FNC_1,False
28,A310,LIS,18:50:00,00:50:00,A310_LIS_0,True
29,A310,LIS,00:50:00,06:30:00,A310_LIS_1,False
30,A310,LIS,06:30:00,08:05:00,A310_LIS_2,False
31,A310,LIS,08:05:00,11:00:00,A310_LIS_3,False
32,A310,LIS,11:00:00,12:10:00,A310_LIS_4,False
33,A310,LIS,12:10:00,14:05:00,A310_LIS_5,False


In [67]:
# Create a nodes_df
nodes_df = ground_arcs[['AC Type', 'Airport', 'Start Time']].rename(columns={'Start Time': 'Time'})

# Add a count number for each row group by AC Type and Airport
nodes_df['Node ID'] = nodes_df.groupby(['AC Type']).cumcount()

# make a dictionary with the ac type as main key and the airport as secondary key with the node as tertiary key and the time as value
nodes = {}
for k in ac_list:
    nodes[k] = {}
    for n in airports:
        nodes[k][n] = {}
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i] = {'Time': nodes_df[(nodes_df['AC Type'] == k) & 
                                               (nodes_df['Airport'] == n) & 
                                               (nodes_df['Node ID'] == i)]['Time'].values[0]}
            

In [68]:
# Add node id to the events
events.merge(nodes_df, how='left', on=['AC Type', 'Airport', 'Time']).sort_values(by=['AC Type', 'Airport'])

# Add a dictionary call departures and another one arrivals to the nodes dictionary at k,n,i+1 with 
# the events that have the same ac type, airport and time as the node
for k in ac_list:
    for n in airports:
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i]['Departures'] = list(events[(events['AC Type'] == k) & (events['Airport'] == n) & (events['Time'] == nodes[k][n][i]['Time']) & (events['D_A'] == 'Departure')]['Flight N'])
            nodes[k][n][i]['Arrivals'] = list(events[(events['AC Type'] == k) & (events['Airport'] == n) & (events['Time'] == nodes[k][n][i]['Time']) & (events['D_A'] == 'Arrival')]['Flight N'])


In [69]:
# n+: ground arcs originating at any node n (start time)
# n-: ground arcs ending at any node n (end time)
n_plus = ground_arcs[['Airport', 'AC Type', 'Start Time', 'Arc ID']].rename(columns={'Start Time': 'Time'})
n_minus = ground_arcs[['Airport', 'AC Type', 'End Time', 'Arc ID']].rename(columns={'End Time': 'Time'})

# Add a dictionary call n+ and another one n- to the nodes dictionary at k,n,i+1 with 
# the events that have the same ac type, airport and time as the node
for k in ac_list:
    for n in airports:
        for i in nodes_df[(nodes_df['AC Type'] == k) & (nodes_df['Airport'] == n)]['Node ID']:
            nodes[k][n][i]['n+'] = list(n_plus[(n_plus['AC Type'] == k) & (n_plus['Airport'] == n) & (n_plus['Time'] == nodes[k][n][i]['Time'])]['Arc ID'])
            nodes[k][n][i]['n-'] = list(n_minus[(n_minus['AC Type'] == k) & (n_minus['Airport'] == n) & (n_minus['Time'] == nodes[k][n][i]['Time'])]['Arc ID'])

In [70]:
overnight_arcs = ground_arcs[ground_arcs['Overnight'] == True][['AC Type', 'Airport', 'Arc ID']]
overnight_flights = []
for l in flights:
    for k in flights[l]:
        if flights[l][k]['Overnight']:
            overnight_flights.append([k, l])

overnight_flights = pd.DataFrame(overnight_flights, columns=['AC Type', 'Flight no.'])

In [71]:
overnight_flights

,AC Type,Flight no.
0,A310,S4128
1,A320,S4128


In [72]:
import random
import plotly.graph_objects as go

for k in ac_list:
    df = events[events['AC Type'] == k].sort_values(by=['Time'])
    fig = go.Figure()
    
    # Set marker color based on 'D_A' column
    marker_color = ['red' if d_a == 'Departure' else 'blue' for d_a in df['D_A']]
    
    # Add ground arcs
    ground_arcs_k = ground_arcs[ground_arcs['AC Type'] == k]
    for i, row in ground_arcs_k.iterrows():
        # Generate a random color
        random_color = '#' + ''.join(random.choices('0123456789ABCDEF', k=6))
        fig.add_shape(
            type="line",
            x0=row['Start Time'],
            y0=row['Airport'],
            x1=row['End Time'],
            y1=row['Airport'],
            line=dict(color=random_color, width=2)
        )
    
    fig.add_trace(go.Scatter(
        x=df['Time'], 
        y=df['Airport'], 
        mode='markers+text',
        marker=dict(color=marker_color),
        hovertemplate= '<b>Flight no.</b>: ' + df['Flight N'] + '<br>' ))
    
    
    fig.update_layout(
        title="Ground Arcs for AC Type " + k,
        xaxis_title="Time",
        yaxis_title="Airport",
    )
    fig.show()


In [16]:
# Define the model
m = gp.Model('FAM')

# Define the decision variables
# f[i,k] 1 if flight arc i is assigned to aircraft type k, 0 otherwise
f = {}

for i in flights:
    for k in flights[i]:
        f[i, k] = m.addVar(vtype=GRB.BINARY, name='f_' + str(i) + '_' + str(k))

# y_ak = number of aircraft of type k on the ground arc a (integer with lowerbound 0)
y = {}
for k in ac_list:
    for a in list(ground_arcs[(ground_arcs['AC Type'] == k)]['Arc ID']):
        y[a, k] = m.addVar(vtype=GRB.INTEGER, name='y_' + str(a) + '_' + str(k))

m.update()

# Define the objective function
m.setObjective(gp.quicksum(aircraft[k]['CASK']/100 *        # Cost per Available Seat Kilometer
                           aircraft[k]['Seats'] *       # Number of seats
                           flights[i][k]['Distance'] *     # Distance
                           f[i,k]                       # Binary variable
                           for i in flights for k in flights[i]), GRB.MINIMIZE)


# Define the constraints
# Constraint 1: Each flight is assigned to exactly one aircraft type
for i in flights:
    m.addConstr((gp.quicksum(f[i,k] for k in flights[i]) == 1), name='one_ac')

# Constraint 2: The number of AC arriving = AC departing, for each type at each node
# y_n+_k + sum(f_i,k) = y_n-_k + sum(f_i,k)
for k in ac_list:
    for n in airports:
        for i in nodes[k][n]:
            m.addConstr((y[nodes[k][n][i]['n+'][0], k] + gp.quicksum(f[w,k] for w in nodes[k][n][i]['Departures']) == 
                         y[nodes[k][n][i]['n-'][0], k] + gp.quicksum(f[w,k] for w in nodes[k][n][i]['Arrivals']) ),
                         name='balance_' + str(i) + '_' + str(k) + '_' + str(n))

# Constraint 3: The number of overnight arcs + the number of overnight flights = the number of aircraft of each type 
# using overnight_arcs and overnight_flights
# sum(y_a,k) + sum(f_i,k) = number of aircraft of type k
for k in ac_list:
    m.addConstr((gp.quicksum(y[a, k] for a in list(overnight_arcs[(overnight_arcs['AC Type'] == k)]['Arc ID'])) #+ 
                 #gp.quicksum(f[i,k] for i in list(overnight_flights[(overnight_flights['AC Type'] == k)]['Flight no.'])) 
                 == aircraft[k]['Fleet']), name='overnight_' + str(k))

m.update()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-19


In [17]:
# Optimize the model but dont print the output
m.Params.OutputFlag = 0
m.optimize()
print('Optimal solution: %0.1f' % (m.objVal / 1000))

Optimal solution: 437.7


In [18]:
for k in ac_list:
    for n in airports:
        for i in nodes[k][n]:
            # val = 0
            s = str(i) + " | " + str(k) + " | "  + str(n)+ " | " +str(nodes[k][n][i]['Time']) + " | " 
            s += str(y[nodes[k][n][i]['n+'][0], k].varName) + ' (n+)'
            # val += y[nodes[k][n][i]['n+'][0], k].varName
            for w in nodes[k][n][i]['Departures']:
                s += ' + ' + str(f[w, k].varName) + ' (f+)'
                # val += f[w, k].varName
            s += ' = ' + str(y[nodes[k][n][i]['n-'][0], k].varName)  + ' (n-)'
            # val -= y[nodes[k][n][i]['n-'][0], k].varName
            for w in nodes[k][n][i]['Arrivals']:
                s += ' + ' + str(f[w, k].varName) + ' (f-)'
                # val -= f[w, k].varName
            # s += ' = ' + str(val)
            print(s)

31 | A310 | YTO | 02:45:00 | y_31_A310 (n+) + f_S4223_A310 (f+) = y_32_A310 (n-)
32 | A310 | YTO | 00:25:00 | y_32_A310 (n+) = y_31_A310 (n-) + f_S4222_A310 (f-)
2 | A310 | FNC | 17:05:00 | y_2_A310 (n+) + f_S4161_A310 (f+) = y_3_A310 (n-)
3 | A310 | FNC | 13:25:00 | y_3_A310 (n+) = y_2_A310 (n-) + f_S4160_A310 (f-)
11 | A310 | OPO | 21:50:00 | y_11_A310 (n+) = y_14_A310 (n-) + f_S4212_A310 (f-)
12 | A310 | OPO | 12:25:00 | y_12_A310 (n+) = y_11_A310 (n-) + f_S4320_A310 (f-)
13 | A310 | OPO | 13:30:00 | y_13_A310 (n+) + f_S4321_A310 (f+) = y_12_A310 (n-)
14 | A310 | OPO | 19:05:00 | y_14_A310 (n+) + f_S4213_A310 (f+) = y_13_A310 (n-)
0 | A310 | BOS | 22:25:00 | y_0_A310 (n+) = y_1_A310 (n-) + f_S4221_A310 (f-)
1 | A310 | BOS | 03:15:00 | y_1_A310 (n+) + f_S422X_A310 (f+) = y_0_A310 (n-)
15 | A310 | PDL | 22:05:00 | y_15_A310 (n+) + f_S4128_A310 (f+) = y_30_A310 (n-) + f_S4213_A310 (f-)
16 | A310 | PDL | 08:40:00 | y_16_A310 (n+) = y_15_A310 (n-) + f_S422X_A310 (f-)
17 | A310 | PDL | 09

In [19]:
for k in ac_list:
    for n in airports:
        for i in nodes[k][n]:
            val = 0
            s = str(i) + " | " + str(k) + " | "  + str(n)+ " | " +str(nodes[k][n][i]['Time']) + " | " 
            s += str(y[nodes[k][n][i]['n+'][0], k].x) + ' (n+)'
            val += y[nodes[k][n][i]['n+'][0], k].x
            for w in nodes[k][n][i]['Departures']:
                s += ' + ' + str(f[w, k].x) + ' (f+)'
                val += f[w, k].x
            s += ' = ' + str(y[nodes[k][n][i]['n-'][0], k].x)  + ' (n-)'
            val -= y[nodes[k][n][i]['n-'][0], k].x
            for w in nodes[k][n][i]['Arrivals']:
                s += ' + ' + str(f[w, k].x) + ' (f-)'
                val -= f[w, k].x
            s += ' = ' + str(val)
            print(s)

31 | A310 | YTO | 02:45:00 | -0.0 (n+) + 1.0 (f+) = 1.0 (n-) = 0.0
32 | A310 | YTO | 00:25:00 | 1.0 (n+) = -0.0 (n-) + 1.0 (f-) = 0.0
2 | A310 | FNC | 17:05:00 | -0.0 (n+) + 1.0 (f+) = 1.0 (n-) = 0.0
3 | A310 | FNC | 13:25:00 | 1.0 (n+) = -0.0 (n-) + 1.0 (f-) = 0.0
11 | A310 | OPO | 21:50:00 | 0.0 (n+) = 0.0 (n-) + -0.0 (f-) = 0.0
12 | A310 | OPO | 12:25:00 | 0.0 (n+) = 0.0 (n-) + -0.0 (f-) = 0.0
13 | A310 | OPO | 13:30:00 | 0.0 (n+) + -0.0 (f+) = 0.0 (n-) = 0.0
14 | A310 | OPO | 19:05:00 | 0.0 (n+) + -0.0 (f+) = 0.0 (n-) = 0.0
0 | A310 | BOS | 22:25:00 | 3.0 (n+) = 2.0 (n-) + 1.0 (f-) = 0.0
1 | A310 | BOS | 03:15:00 | 2.0 (n+) + 1.0 (f+) = 3.0 (n-) = 0.0
15 | A310 | PDL | 22:05:00 | 1.0 (n+) + -0.0 (f+) = 1.0 (n-) + -0.0 (f-) = 0.0
16 | A310 | PDL | 08:40:00 | 2.0 (n+) = 1.0 (n-) + 1.0 (f-) = 0.0
17 | A310 | PDL | 09:00:00 | 3.0 (n+) = 2.0 (n-) + 1.0 (f-) = 0.0
18 | A310 | PDL | 09:25:00 | 3.0 (n+) + -0.0 (f+) = 3.0 (n-) + -0.0 (f-) = 0.0
19 | A310 | PDL | 09:35:00 | 3.0 (n+) + -0.0 (

In [20]:
print('Flights assigned to aircraft type: A320')
for i in flights:
    for k in flights[i]:
        if f[i,k].x > 0 and k == 'A320':
            print(i, k, ('Cost %0.2f' % (aircraft[k]['CASK'] *  aircraft[k]['Seats'] *  flights[i][k]['Distance'] * f[i,k].x / 1000000)))
print('\n')

print('Flights assigned to aircraft type: A310')
for i in flights:
    for k in flights[i]:
        if f[i,k].x > 0 and k == 'A310':
            print(i, k, ('Cost %0.2f' % (aircraft[k]['CASK'] *  aircraft[k]['Seats'] *  flights[i][k]['Distance'] * f[i,k].x / 1000000)))            

Flights assigned to aircraft type: A320
S4120 A320 Cost 1.10
S4220 A320 Cost 1.10
S4124 A320 Cost 1.10
S4128 A320 Cost 1.10
S4320 A320 Cost 1.16
S4212 A320 Cost 1.16
S4121 A320 Cost 1.10
S422Y A320 Cost 1.10
S4125 A320 Cost 1.10
S4129 A320 Cost 1.10
S4321 A320 Cost 1.16
S4213 A320 Cost 1.16


Flights assigned to aircraft type: A310
S4160 A310 Cost 1.57
S4222 A310 Cost 7.32
S4221 A310 Cost 6.26
S4161 A310 Cost 1.57
S4223 A310 Cost 7.32
S422X A310 Cost 6.26


In [21]:
for v in m.getVars():
    for k in ac_list:
        if v.x != 0 and v.varName[0] == 'y'and v.varName[-4:] == k:
            print('%s %g' % (v.varName, v.x))

y_0_A310 3
y_1_A310 2
y_3_A310 1
y_15_A310 1
y_16_A310 2
y_17_A310 3
y_18_A310 3
y_19_A310 3
y_20_A310 2
y_21_A310 2
y_22_A310 2
y_23_A310 2
y_24_A310 1
y_25_A310 1
y_26_A310 1
y_29_A310 1
y_30_A310 1
y_32_A310 1
y_2_A320 2
y_3_A320 3
y_4_A320 2
y_5_A320 1
y_7_A320 1
y_8_A320 2
y_9_A320 3
y_10_A320 1
y_11_A320 2
y_12_A320 1
y_14_A320 1
y_15_A320 2
y_16_A320 1
y_19_A320 1
y_21_A320 1
y_23_A320 1
y_26_A320 1
y_27_A320 2


In [22]:
# Make a df with AC grounded at overnight on each airport
grounded_overnight = pd.DataFrame(columns=ac_list, index=airports)
overnight_arcs['count'] = overnight_arcs.apply(lambda row: y[row['Arc ID'], row['AC Type']].x, axis=1)

# Reorganize the df, make the columns the unique AC types and the rows the unique airports and the values the count
for k in ac_list:
    grounded_overnight[k] = overnight_arcs[overnight_arcs['AC Type'] == k].groupby(['Airport']).sum()['count']

# Fill the NaN values with 0
grounded_overnight.fillna(0, inplace=True)
grounded_overnight

,A310,A320
YTO,0.0,0.0
FNC,0.0,0.0
OPO,0.0,1.0
BOS,3.0,0.0
PDL,1.0,1.0
LIS,0.0,2.0


In [23]:
nodes


{'A310': {'YTO': {31: {'Time': datetime.time(2, 45),
    'Departures': ['S4223'],
    'Arrivals': [],
    'n+': [31],
    'n-': [32]},
   32: {'Time': datetime.time(0, 25),
    'Departures': [],
    'Arrivals': ['S4222'],
    'n+': [32],
    'n-': [31]}},
  'FNC': {2: {'Time': datetime.time(17, 5),
    'Departures': ['S4161'],
    'Arrivals': [],
    'n+': [2],
    'n-': [3]},
   3: {'Time': datetime.time(13, 25),
    'Departures': [],
    'Arrivals': ['S4160'],
    'n+': [3],
    'n-': [2]}},
  'OPO': {11: {'Time': datetime.time(21, 50),
    'Departures': [],
    'Arrivals': ['S4212'],
    'n+': [11],
    'n-': [14]},
   12: {'Time': datetime.time(12, 25),
    'Departures': [],
    'Arrivals': ['S4320'],
    'n+': [12],
    'n-': [11]},
   13: {'Time': datetime.time(13, 30),
    'Departures': ['S4321'],
    'Arrivals': [],
    'n+': [13],
    'n-': [12]},
   14: {'Time': datetime.time(19, 5),
    'Departures': ['S4213'],
    'Arrivals': [],
    'n+': [14],
    'n-': [13]}},
  'BOS': {